### Similitud entre productos

Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen.
Para este desafío tenemos un dataset “items_titles.csv” que tiene títulos de 30 mil productos de 3 categorías diferentes de Mercado Libre Brasil

El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset “item_titles_test.csv”

In [1]:
# Libraries
import sys 
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_rows', None)

In [2]:
# Parameters
NAME_FILE_TEST = "items_titles_test.csv"
FILES_PATH = '../Files/'

In [3]:
# Open csv file and view it of test
df_test = pd.read_csv(FILES_PATH+NAME_FILE_TEST)
df_test.columns = df_test.columns.str.lower()
print("Number of rows: "+str(df_test.shape[0]))
df_test.head()

Number of rows: 10000


,ite_item_title
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


To execute this task, we'll use BERT model.

### Auxiliary function

In [4]:
# Function that creates the similarity score
def similarity_score(dataframe: pd.DataFrame, item_title:str) -> pd.DataFrame:
    """It receives a title and delivers a dataframe with the similarity score, comparing it with the rest of the items.
 
    Args:
        dataframe (pd.DataFrame): Dataframe with the titles
        item_title (str): Item title to use as base to compare it with the rest of items.

    Returns:
        result_df (pd.DataFrame): Dataframe with similarities scores

    """
    # Initialize BERT
    sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
    # Transform the titles column into a list of strings
    titles = dataframe['ite_item_title'].values.tolist()
    # Create the embeddings of the titles
    titles_embeddings = sbert_model.encode(titles)
    # Create the similarity matrix using cosine similarity
    cos_similarity = cosine_similarity(titles_embeddings)
    # Create a new DataFrame for storing results
    result_df = pd.DataFrame(columns=['ite_item_title 1', 'ite_item_title 2', 'Similarity Score'])
    # Search the index where the item_title is stored.
    idx = dataframe[dataframe['ite_item_title'] == item_title].index[0]
    # Create a loop that search within the cosine similarity matrix and retrieves the information related to the item_title
    for j in range(idx+1, len(dataframe)):
        title_1 = dataframe['ite_item_title'].iloc[idx]
        title_2 = dataframe['ite_item_title'].iloc[j]
        similarity_score = cos_similarity[idx, j]
        # Add the data to the result dataframe
        result_df = result_df.append(
            {'ite_item_title 1': title_1,
             'ite_item_title 2': title_2,
             'Similarity Score': similarity_score},
             ignore_index=True)
    # Order by similarity score
    result_df = result_df.sort_values('Similarity Score', ascending=False)
    # Return 
    return result_df

In [5]:
# Example of execution
title = 'Tênis Olympikus Esporte Valente - Masculino Kids'
result_df = similarity_score(df_test, title)
result_df = result_df.reset_index(drop = True)
result_df.head(20)

,ite_item_title 1,ite_item_title 2,Similarity Score
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Masculino Bebê Kids Xadrez/vermelho Boti...,0.902779
1,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Liberte Kids Fluidfit Preto Rosa,0.876427
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tenis Infantil Meninas Slip On Kids Molekinha ...,0.874976
3,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Esportivo Infantil Olympikus Azul Marinh...,0.870854
4,Tênis Olympikus Esporte Valente - Masculino Kids,Bicicleta Aro 12 Infantil Clubinho Salva Vidas...,0.866669
5,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Infantil Masculino Olympikus Enjoy Kids ...,0.865717
6,Tênis Olympikus Esporte Valente - Masculino Kids,Tenis Infantil Masculino Molekinho Baby Kids M...,0.864453
7,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis By Kids Carro Infantil Chk Elas,0.851462
8,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Infantil Calce Fácil La Kids Super Leve ...,0.851429
9,Tênis Olympikus Esporte Valente - Masculino Kids,Sapatênis Infantil Bebê Kids Grafite Carrinho ...,0.850661
